In [1]:
#Library
import requests
import json
from datetime import datetime
import math
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 1000)
pd.set_option("display.precision", 12)
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_data(endpointApi, date=None, full_response=False, columns=list()):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    payload = {'date': date, 'fullResponse': full_response}
    res = requests.get(URL + endpointApi,# + columns_to_str(columns),
                      params=payload,
                      auth=AUTH,
                      verify=False)
    return res.content.decode('utf-8')

In [3]:
def put_data(endpointApi, content):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    res = requests.put(url=URL + endpointApi,# + columns_to_str(columns),
                      data=json.dumps(content),
                      auth=AUTH,
                      headers = {"content-type": "application/json"},
                      verify=False)
    return res.content.decode('utf-8')

In [4]:
def post_data(endpointApi, content):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    res = requests.post(url=URL + endpointApi,# + columns_to_str(columns),
                      data=json.dumps(content),
                      auth=AUTH,
                      headers = {"content-type": "application/json"},
                      verify=False)
    return res.content.decode('utf-8')

Conversion chaque valeur en le type approprié :

In [5]:
def convert_currency(value, df_convert_currency):
    value, currency_name = value.split(' ')
    rate = currencyRateToEUR(currency_name, df_currency_rate)
    
    new_value = float(value.replace(',', '.')) * rate
    return new_value

def convert_datetime(value):
    return datetime.strptime(value, '%Y-%m-%d').date()

def convert_float(value):
    return float(value.replace(',', '.'))

def convert_int(value):
    return int(value)

def convert_boolean(value):
    return json.loads(value)

def df_convert_type(df, df_currency_rate):
        for col in df.columns:
            convert_values = []
            for elt in df[col]:
                if elt is np.nan:
                    convert_values.append(np.nan)
                    continue

                elt_type = elt['type']
                elt_value = elt['value']
                if elt_type == 'currency_value':
                    elt_value = convert_currency(elt_value, df_currency_rate)
                elif elt_type == 'date':
                    elt_value = convert_datetime(elt_value)
                elif elt_type in ['double', 'percent']:
                    elt_value = convert_float(elt_value)
                elif elt_type in ['asset', 'int32']:
                    elt_value = convert_int(elt_value)
                elif elt_type == 'boolean':
                    elt_value = convert_boolean(elt_value)
                elif elt_type not in ['asset_type', 'string', 'asset_currency', 'date-time',
                                     'asset_sub_type', 'asset_status', 'asset_quote_type',
                                     'liquidity_algorithm', 'portfolio_lock_mode', 'portfolio_type']:
                    print(elt)
                convert_values.append(elt_value)
            df[col] = convert_values
        return df

Création de la table des taux en EUR : 
 
[Est-ce qu'on donne la posibilité de changer la currency dst en fonction de la currency du portfolio ?]

In [6]:
def getCurrencyRate():
    d = []
    currency_json = json.loads(get_data('currency'))
    for currency in currency_json:
        currency_id = currency.get('id')
        rate = get_data('currency/rate/{}/to/EUR'.format(currency_id))
        if len(rate) == 0:
            continue
        rate_value = json.loads(rate)['rate']['value']
        d.append([currency_id, convert_float(rate_value)])
    return pd.DataFrame(d, columns=['currency src', 'rate'])
    
df_currency_rate = getCurrencyRate()
df_currency_rate

,currency src,rate
0,USD,0.851208716377
1,EUR,1.000000000000
2,JPY,0.008066467694
3,BEF,0.024789352477
4,CYP,1.708601441376
...,...,...
23,LUF,0.024789352477
24,VAL,0.000516456899
25,SML,0.000516456899
26,EUr,0.010000000000


[TEST] Taux de convertion entre USD et EUR :

In [7]:
def currencyRateToEUR(cur_src, df_currency_rate):
    return df_currency_rate[df_currency_rate['currency src'] == cur_src]['rate'].values[0]
rate = currencyRateToEUR('USD', df_currency_rate)
rate


0.851208716377

[TEST] La database complète de tous les asset :

In [8]:
data = get_data('asset/')
df = pd.read_json(data)
df_assets = df_convert_type(df, df_currency_rate)
df_assets

,CURRENCY,MARKET_PLACE_MIC,SUB_SECTOR,ISIN_CODE,LAST_GROSS_VALUE,LAST_QUOTE_DATE,MARKET_PLACE_CURRENCY,ASSET_DATABASE_ID,CATEGORY_CIC,CREATION_DATE,ASSET PRICE MODE,FIRST_QUOTE_DATE,LAST_QUOTE_SOURCE,VL_DELAY,IS_ACCESSIBILITY_PUBLIC,SUB_CATEGORY_CIC,MARKET_CAP_CATEGORY,CODE_CIC,INDEXED_RATE,GRAPH_PAST_WEEK,GRAPH_LAST_YEAR,CITY_OR_ACRONYM_OR_NAME,PORT_LINE_COUNT,PAY_ACCOUNT_ON_DEFAULT,MARKET_PLACE_CITY,IS_PORTFOLIO,DAILY_PRICE_COUNT,ASSET_STATUS,LAST_QUOTES_UPDATE_DATE,MARKET_PLACE_DESC,EMIR_CLEARING_ID_TYPE,GRAPH_PERIOD,INDUSTRY,IS_STRUCTURED_PRODUCT,BLOOMBERG_CODE,GRAPH_PAST_YEAR,SEDOL_CODE,LIQUIDITY_ALGO,asset_custom_classification_bloomberg,COUNTRY_CIC,GLOBAL_TYPE,ASSET_OR_MARKET_PLACE_COUNTRY,LAST_CLOSE_VALUE_IN_CURR,MARKET_PLACE,LAST_CLOSE_VALUE,MODIFICATION_DATE,GICS_INDUSTRY_GROUP,PORT_CONTENT_COUNT,IS_FUND_FED,IS_PUBLICLY_TRADED,COUNTRY_ISO_3,FLOATING_VALUE,FINAL_UNDERLYING,UNDERLYING_OR_ASSET_IF_UNDERLYING_NOT_AVAILABLE,GICS_SECTOR,MARKET_PLACE_TELEKURS,GRAPH_LAST_6_MONTH,asset_attr_init,MARKET_PLACE_OR_ASSET_COUNTRY,SUB_TYPE,MARKET_PLACE_COUNTRY,TYPE,LABEL,REST_OBJECT_ID,IS_EURONEXT,IS_COMPOSITE,FINAL_UNDERLYING_TYPE,IS_FUND_SAVE_SENS,GICS_INDUSTRY,REST_UNIQUE_NUMBER,STOCK_TYPE,GICS_SUB_INDUSTRY,SECTOR,quote_value_type,QUOTE_TYPE,IS_EUROLAND,MARKET_PLACE_TYPE,BLOOMBERG_ID,SUP_SECTOR,IS_DEPOSITARY_QUOTES,COUNTRY,asset_custom_classification_Tab. de bord GSM,ASSET_MARKET_SUB_GROUP,ELIGIBLE_PEA_PME,ELIGIBLE_PEA,BOND_WAL_COMPUTATION_MODE,asset_fund_info_asset_class_focus,asset_fund_info_strategy,asset_fund_info_maximum_front_end_fee,asset_fund_info_bloomberg_ticker,asset_fund_info_is_feeded,asset_fund_info_authorized_for_retail_investors,TRANSPARENCY_LIST,asset_fund_info_fund_leverage,asset_fund_info_objective_long,asset_fund_info_is_euro_ucits,asset_fund_info_administrator,BOND_CREDIT_RISK_COMPUTATION_MODE,asset_fund_info_isin_code,asset_fund_info_minimum_investment_shares,BOND_WAM_COMPUTATION_MODE,asset_fund_info_depositary,asset_fund_info_manager_stated_fee,BOND_YTM_COMPUTATION_MODE,asset_fund_info_cut_off,asset_fund_info_maximum_back_end_fee,asset_fund_info_company,asset_fund_info_vl_date,IS_REAL_ESTATE_FUND,asset_fund_info_manager_stated_perf_fee,asset_fund_info_auditor,FUND_TYPE,BOND_RISK_COMPUTATION_MODE,asset_fund_info_fund_type,asset_fund_info_valorisation_frequency,asset_fund_info_creation_date,MIN_BUY_INITIAL_AMOUNT,MIN_BUY_AMOUNT,asset_fund_info_pct_max_opcvm,asset_fund_info_initial_minimum_subscription,asset_fund_info_fund_open_pymt_shr,asset_fund_info_amf_classification,asset_fund_info_decimalisation,asset_fund_info_amf_date,asset_fund_info_minimum_subscription,asset_fund_info_day_cut_off,asset_fund_info_vl_known,asset_fund_info_geographical_focus,asset_fund_info_subscription_committee,asset_fund_info_abide_with_the_four_criterias,asset_fund_info_reuters_code,REUTERS_CODE,asset_fund_info_subscription_frequency,asset_fund_info_redemption_frequency,VALOREN_CODE,asset_fund_info_valoren_code,asset_fund_info_manager,asset_fund_info_redemption_committee,asset_fund_info_advised_horizon,FUND_REF_IDX,asset_fund_info_comanager,HAS_TOBIN_TAX,CUSIP_CODE,LOCAL_CODE,asset_fund_info_rate_entry,asset_fund_info_management_style,asset_fund_info_rate_output,asset_fund_info_rate_entry_sdg,asset_fund_info_rate_output_sdg,MARKET_QUOTE_IDENTIFIER,asset_fund_info_manager_committee,PORT_ASSET_CONTENT_COUNT,INDEX_TYPE,TELEKURS_QUOTE_PLACE_CODE,QUOTE_PLACE,IS_SRD,PORT_BASE_NAV,PORT_VL,PORT_RATING_MOODYS_SHORT_TERM,REAL_TIME_PORT_CASH,PORT_STATUS,PORT_FEED_TYPE,PORT_TYPE,PORT_CURRENCY_HEDGE,PORT_PERCENT_EQUIVALENT,PORT_CASH,PORT_NET_EX_CASH_EQUIVALENT,NB_COMPTE_ON_PORTFOLIO,PORT_CLEAN_MODE,PORT_RATING_SP_SHORT_TERM,EAI_FRONTPORT_CASH_BY_CRCY,PORT_IS_HISTORICAL,PRICER_MODE,PORT_CASH_AVAILABLE,EAI_BACKPORT_CASH_BY_CRCY,EAI_PORTFOLIO_FEED_NAME,PORT_RATING_MOODYS,PORT_NET_EX_CASH,PORT_RATING_SP,LOCK_MODE,PORT_PERCENT_CASH,PORT_NEED_RECONCILIATION,PORT_CASH_EQUIVALENTS,PORT_RATING_FITCH,PORT_PERCENT_CASH_AVAILABLE,EAI_PORT_AVAILABLE_CASH,PO

Récupération de la liste d'actifs en base :

[On prend juste ces valeurs comme dans l'exemple ou toutes les valeurs des assets ?]

In [9]:
def get_df_asset(asset_date, df_currency_rate, asset_id='ASSET_DATABASE_ID', asset_label='LABEL',
                 asset_type='TYPE', asset_last_close_value='LAST_CLOSE_VALUE_IN_CURR'):
    data = get_data('asset?columns={}&columns={}&columns={}&columns={}&date={}'
                   .format(asset_id, asset_label, asset_type, asset_last_close_value, asset_date))
    df_asset = pd.read_json(data)
    df_asset = df_convert_type(df_asset, df_currency_rate)
    return df_asset

df_ETF = get_df_asset('2016-06-01', df_currency_rate)
df_ETF

,ASSET_DATABASE_ID,LABEL,LAST_CLOSE_VALUE_IN_CURR,TYPE
0,1845,1&1 DRILLISCH AG,36.851000000000,STOCK
1,1846,1000MERCIS,40.900000000000,STOCK
2,2122,1818-VEGA EUR CO,39.370000000000,FUND
3,2123,29 HAUSSMANN CROISSANCE EUROPE D,1670.650000000000,FUND
4,2124,29 HAUSSMANN EQUILIBRE,NaN,FUND
...,...,...,...,...
475,1879,YAMANA GOLD INC,3.780000000000,STOCK
476,1997,YMAGIS SA,9.460000000000,STOCK
477,1991,ZCCM INVESTMEN - B,1.000000000000,STOCK
478,2106,ZOETIS INC,40.730337078639,STOCK


Différence entre LAST_CLOSE_VALUE_IN_CURR et LAST_CLOSE_VALUE ??

Récupération des valeurs de cotation:

Cotation de l'actif 1845 en fonction de l'id et de la date de debut et fin :

In [10]:
def get_cotation_values(asset_id, start_date, end_date, df_currency_rate):
    data = get_data('asset/{}/quote?start_date={}&end_date={}'
                    .format(asset_id, start_date, end_date))
    df_cotation = pd.read_json(data)
    df_cotation = df_convert_type(df_cotation, df_currency_rate)
    return df_cotation

df_cotation = get_cotation_values('1845', '2016-06-01', '2020-09-30', df_currency_rate)
df_cotation
    

,date,nav,gross,volume,high,real_close_price,feed_source,low,ask,asset,pl,bid,close,return,open
0,2016-06-01,36.851000000000,36.851000000000,143199.0,37.125,36.851000000000,-10.0,36.675,36.799,1845,0.037000000000,36.723,36.851000000000,0.001005052426,36.745
1,2016-06-02,36.900000000000,36.900000000000,147480.0,37.145,36.900000000000,-10.0,36.594,36.999,1845,0.049000000000,36.946,36.900000000000,0.001329678978,36.786
2,2016-06-03,36.052000000000,36.052000000000,190934.0,37.200,36.052000000000,-10.0,35.800,36.277,1845,-0.848000000000,36.219,36.052000000000,-0.022981029810,36.900
3,2016-06-06,36.540000000000,36.540000000000,82279.0,36.775,36.540000000000,-10.0,36.082,36.587,1845,0.488000000000,36.534,36.540000000000,0.013536003550,36.190
4,2016-06-07,37.246000000000,37.246000000000,119105.0,37.347,37.246000000000,-10.0,36.470,37.172,1845,0.706000000000,37.150,37.246000000000,0.019321291735,36.590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,2020-09-24,27.091248954261,27.091248954261,NaN,NaN,27.091248954261,-10.0,NaN,NaN,1845,-0.550081510430,NaN,27.091248954261,-0.019900688613,NaN
1117,2020-09-25,26.877384954417,26.877384954417,NaN,NaN,26.877384954417,-10.0,NaN,NaN,1845,-0.213863999844,NaN,26.877384954417,-0.007894209684,NaN
1118,2020-09-28,26.634573242213,26.634573242213,NaN,NaN,26.634573242213,-10.0,NaN,NaN,1845,-0.242811712203,NaN,26.634573242213,-0.009034052703,NaN
1119,2020-09-29,26.368478372669,26.368478372669,NaN,NaN,26.368478372669,-10.0,NaN,NaN,1845,-0.266094869544,NaN,26.368478372669,-0.009990581307,NaN


Clean le dataframe df_ETF pour garder seulement les actif qui représentent un %NAV du portefeuille entre 1 et 10% à la date du 01/06/2016 :



et si l'asset n'a pas de nav à la date du 01/06/2016, on fait quoi ? on prend la valeur du jour d'avant ?

In [11]:
"""def keep_only_asset_with_suitable_nav(df_asset, date, nav_min, nav_sup, df_currency_rate):
    assets_to_keep = []
    for index, row in df_asset.iterrows():
        asset_id = row['ASSET_DATABASE_ID']
        df_cot = get_cotation_values(asset_id, date, date, df_currency_rate)
        if df_cot.empty:
            continue
        nav = df_cot.loc[df_cot['asset'] == asset_id]['nav'][0]
        if 1 <= nav and nav <= 10:
            assets_to_keep.append(asset_id)
    df_asset = df_asset.loc[df_asset['ASSET_DATABASE_ID'].isin(assets_to_keep)]
    return df_asset

df_ETF = keep_only_asset_with_suitable_nav(df_ETF, '2016-06-01', 1, 10, df_currency_rate)
df_ETF"""

"def keep_only_asset_with_suitable_nav(df_asset, date, nav_min, nav_sup, df_currency_rate):\n    assets_to_keep = []\n    for index, row in df_asset.iterrows():\n        asset_id = row['ASSET_DATABASE_ID']\n        df_cot = get_cotation_values(asset_id, date, date, df_currency_rate)\n        if df_cot.empty:\n            continue\n        nav = df_cot.loc[df_cot['asset'] == asset_id]['nav'][0]\n        if 1 <= nav and nav <= 10:\n            assets_to_keep.append(asset_id)\n    df_asset = df_asset.loc[df_asset['ASSET_DATABASE_ID'].isin(assets_to_keep)]\n    return df_asset\n\ndf_ETF = keep_only_asset_with_suitable_nav(df_ETF, '2016-06-01', 1, 10, df_currency_rate)\ndf_ETF"

Il ne reste plus que seulement 71 actifs ??

Récupération de la liste des ratios :

In [12]:
def get_ratio(df_currency_rate):
    data = get_data('ratio')
    df_ratio = pd.read_json(data)
    #df_ratio = df_convert_type(df_ratio, df_currency_rate)
    return df_ratio

df_ratio = get_ratio(df_currency_rate)
df_ratio

,id,type,name,is_benchmark_needed,is_percent
0,11,Ratio,Correlation,True,False
1,7,Ratio,Exposition action,False,True
2,15,Ratio,Exposition action,False,True
3,8,Ratio,RDT,False,True
4,13,Ratio,Rendement,False,True
5,9,Ratio,Rendement_annualise,False,True
6,12,Ratio,Sharpe,False,False
7,14,Ratio,VaR historique,False,True
8,10,Ratio,Volatilite,False,True


Calcul de ratios :

In [13]:
def post_ratio(ratios, assets_id, start_date, end_date, bench=None, frequency=None):
    start_date = datetime.strptime(start_date, "%Y-%m-%d").isoformat()
    end_date = datetime.strptime(end_date, "%Y-%m-%d").isoformat()
    
    payload = {'ratio':ratios, 'asset':assets_id, 'startDate':[start_date], 'endDate':[end_date], 'bench':bench, 'frequency':frequency}
    reponse = post_data('ratio/invoke', payload)
    return reponse
    
#get Rendement_annualise, Sharpe, Volatilite de l'asset 1845 entre '2016-06-01' et '2020-09-30'
post_ratio([9, 12, 10], [1845], '2016-06-01', '2020-09-30')
#Comment récupérer la Correlation ? (is_benchmark_needed = True)

'{"1845":{"9":{"type":"percent","value":"0,028684656593"},"12":{"type":"double","value":"0,076707259279"},"10":{"type":"percent","value":"0,308766820977"}}}'

Recupération du portfolio ID :

A quoi sert le portfolio LABEL = REF ???

In [14]:
#REST_OBJECT_ID = 1829
#TYPE = PORTFOLIO
#LABEL = EPITA_PTF_10
#LABEL = REF
def get_portfolio_id(df_assets):
    portfolio_id = df_assets.loc[(df_assets['TYPE'] == 'PORTFOLIO') & (df_assets['LABEL'] == 'EPITA_PTF_10')]['REST_OBJECT_ID'].values[0]
    return int(portfolio_id)

portfolio_id = get_portfolio_id(df_assets)
portfolio_id

1829

Récupération de la composition d’un portefeuille : 

In [15]:
def get_portfolio(portfolio_id):
    data = get_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id))
    df_portfolio = pd.read_json(data)
    return df_portfolio
    #keep seulement list assets ?

df_portfolio = get_portfolio(portfolio_id)
df_portfolio

,label,currency,type,values
code,EPITA_PTF_10,EUR,front,NaN


Get assets from portfolio :

In [16]:
def get_assets(df_portfolio, date):
    if date not in df_portfolio['values']:
        return np.NaN
    return df_portfolio['values'][date]

assets = get_assets(df_portfolio, '2016-06-01')
assets

nan

Add asset to portfolio

In [17]:
def add_asset_to_portfolio(df_portfolio, date, asset):
    assets = get_assets(df_portfolio, date)
    if assets is np.nan:
        assets = []
    assets.append(asset)
    return assets

assets = add_asset_to_portfolio(df_portfolio, '2016-06-01', {'asset': {'asset': 1851, 'quantity': 8221}})
assets

[{'asset': {'asset': 1851, 'quantity': 8221}}]

In [18]:
print(df_portfolio['values'].values)

[nan]


MAJ de la composition d’un portefeuille :


In [19]:
def portfolio_json_assets(assets):
    json = ''
    for asset in assets:
        json += '{{"asset":{{"asset": {}, "quantity": {}}}}},'.format(asset["asset"]["asset"], asset["asset"]["quantity"])
    return json[:-1]
    
def seralize_portfolio_content(df_portfoliot, assets):
    label = df_portfoliot['label'][0]#"EPITA_PTF_10"
    currency = df_portfoliot['currency'][0]#"EUR"
    type_ = df_portfoliot['type'][0]
    date = '2016-06-01' #osef de df_portfolio['date'][0] car cf fiche technique : 
    #"La date de la composition. Pour rappel votre composition doit être unique et doit dater du 2013-06-14"
    assets = portfolio_json_assets(assets) 
    res = '{{"label": "{}", "currency": {{"code": "{}"}}, "type": "{}", "values": {{"{}": [{}]}}}}'.format(label, currency, type_, date, assets)
    return json.loads(res)

def deseralize_portfolio_content(json_portfolio):
    json_portfolio = pd.json_normalize(json_portfolio)
    
    cols = [col for col in json_portfolio.columns if col.split('.')[0] == 'values']
    json_portfolio = json_portfolio[cols]
    json_portfolio.columns = json_portfolio.columns.map(lambda x: x.split(".")[0])

    json_assets = json_portfolio['values'][0]
    assets = []
    for asset in json_assets:
        assets.append(asset)
    return assets


json_portfolio = seralize_portfolio_content(df_portfolio, assets)
print(json_portfolio)

assets = deseralize_portfolio_content(json_portfolio)
assets

{'label': 'EPITA_PTF_10', 'currency': {'code': 'EUR'}, 'type': 'front', 'values': {'2016-06-01': [{'asset': {'asset': 1851, 'quantity': 8221}}]}}


[{'asset': {'asset': 1851, 'quantity': 8221}}]

[A CHECK le JSON de put_portfolio]

In [20]:
def put_portfolio(portfolio_id, df_portfolio, assets):
    content = seralize_portfolio_content(df_portfolio, assets)
    put_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id), content)
    
put_portfolio(portfolio_id, df_portfolio, assets)

In [21]:
df_portfolio = get_portfolio(portfolio_id)
print(df_portfolio['values'].values)
df_portfolio

[nan list([{'asset': {'asset': 1851, 'quantity': 8221.0}}])]


,label,currency,type,values
code,EPITA_PTF_10,EUR,front,NaN
2016-06-01,EPITA_PTF_10,NaN,front,"[{'asset': {'asset': 1851, 'quantity': 8221.0}}]"


Reset asset in portfolio :

In [22]:
def reset_assets_portfolio(portfolio_id, df_portfolio):
    content = seralize_portfolio_content(df_portfolio, {})
    content['values'] = {}
    put_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id), content)

    
reset_assets_portfolio(portfolio_id, df_portfolio)

df_portfolio = get_portfolio(portfolio_id)
df_portfolio

,label,currency,type,values
code,EPITA_PTF_10,EUR,front,NaN


[TODO] Functions :

In [23]:
def volatility(returns, nb_period_per_year): #X : col du df
    return returns.std() * np.sqrt(nb_period_per_year)

volatility(df_cotation['close'].values, 365)

210.77009385378707

In [24]:
def beta(X, Y):
    return np.cov(X, Y) / np.var(Y)

In [25]:
def correlation(Returna, Returnb):
    return np.cov(Returna, Returnb) / np.sqrt(np.var(Returna) * np.var(Returnb))

Chaque actif doit représenter un %NAV du portefeuille entre 1 et 10% à la date du 01/06/2016 :

In [26]:
def rendement_Nav(Nav, Nav_prev):
    return (Nav - Nav_prev) / Nav_prev
    #R - R.shift(1))/R.shift(1)

In [27]:
def rendement_annual():
    

SyntaxError: unexpected EOF while parsing (<ipython-input-27-25ba0d1eb457>, line 2)

On évaluera le sharpe sur la période du 01/06/2016 au 30/09/2020:

In [ ]:
def return_rate(X_end, X_begin, nb_days):
    return (1 + (X_end - X_begin) / X_begin)**(365/nb_days) - 1

In [ ]:
def sharpe_ratio(return_rate, nb_period_per_year, Rf=0.05):
    volatility = volatility(returns, nb_period_per_year)
    return (return_rate - Rf) / volatility

In [ ]:
def value_at_risk(Ra, Threshold):
    

In [ ]:
def action_exposure():

Contraites :

50% d'actions  (en montant pas en quantité)

Le portefeuille doit être exactement composé d’un minimum de 15 actifs, et d’un maximum de 40 actifs.

Attention aux devises :

Vérifier bien la devise dans laquelle est exprimée l’actif.

Avez-vous utilisé le bon taux de conversion pour la devise ?

Avez-vous utilisé le taux dans le bon sens ?

Attention aux quotités:

Votre portefeuille comporte-il bien un nombre entier d’actif ? Ou avez-vous acheté 0.0001 action Total ?

L’actif est-il achetable par unité, dizaine, centaines ? le prix donné est-il un prix par unité, dizaine, centaine ?

Le prix de l’actif est-il exprimé en unité ou en centimes ?

Si vous avez arrondis à postériori les quotités, respectez-vous bien toujours les normes fixées pour le portefeuille ?